In [27]:
#%%
print('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting')

from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
#%%
seed = 42
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# train_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_train_gene.csv', delimiter = ',')
# train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_train_hml.csv')
# train_target = train_target[['EMB_MIC']]
# # don't touch test data, split out validation data from training data during training
# # test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_EMB/aa_data_test_pca4k.csv', delimiter = ',')
# test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_test_gene.csv', delimiter = ',')
# test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_test_hml.csv')
# test_target = test_target[['EMB_MIC']]

# all_data = np.concatenate((train_data, test_data), axis=0)
# all_target = pd.concat((train_target, test_target), axis=0)

# train_data, test_data, train_target, test_target = train_test_split(all_data, all_target, test_size=0.2, random_state=42, stratify=all_target)

/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting


## functions

In [28]:
def data_split(aa_array, encoded_mic):
    # Encode the target variable
    # Perform stratified train-test split
    train_data, test_data, train_target, test_target = train_test_split(
        aa_array,
        encoded_mic,
        test_size=0.1,  # 10% for testing
        stratify=encoded_mic,  # Ensures the proportion of each class is preserved
        random_state=42  # For reproducibility
    )
    return train_data, test_data, train_target, test_target
def is_within_doubling_dilution(pred, target, target_min, target_max):
    _ = np.arange(target_min-1, target_max+2, 1)
    index = [i for i, x in enumerate(_) if x == target][0]
    return (_[index-1] <= pred <= _[index+1])

def transform_array(arr):
    transformed = []
    for item in arr:
        # Only process string items that contain a comma
        if isinstance(item, str) and ',' in item:
            parts = [part.strip() for part in item.split(',')]
            transformed.extend(parts)
        else:
            transformed.append(item)
    return transformed

def data_prep_cryptic(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    print(overlap)
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
    print(len(overlap))
    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
        else:
            aa.append([0]*len(all_snp))
        
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1
    
    mic_aa = cryptic[cryptic['ENA_RUN'].isin(overlap)]#.iloc[:,14:27]
    # mic_aa = cryptic[cryptic['ENA_RUN'].isin(variants['sample_id'].unique())]#.iloc[:,14:27]
    # print(mic_aa.shape)
    # mic_aa['wgs_id'] = pd.Categorical(mic_aa['ENA_RUN'], categories=variants['sample_id'].unique().tolist(), ordered=True)
    # mic_aa = mic_aa.sort_values('ENA_RUN')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)
    # mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(variants['sample_id'].unique().tolist())
    mic_aa = mic_aa.sort_values(["ENA_RUN"])  ## 'sort' changed to 'sort_values'
    
    return aa_array, mic_aa

def data_prep_50k(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    np.save(f'all_snp_{len(all_snp)}.npy', all_snp)
    # np.save('all_snp_emb.npy', all_snp)
### 50k data reload
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = variants[variants['SNP'].isin(all_snp)]
    # certify we want 50k data that doesn't overlap with cryptic
    overlap = variants[~variants['sample_id'].isin(cryptic['ENA_RUN'])]
    overlap = overlap['sample_id'].unique()

    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
            # print(variants[variants['sample_id']==x]['drugs'].unique())
            result_ = transform_array(variants[variants['sample_id']==x]['drugs'].unique())
            # print(result_)
            # print(dr_list[0])
            if dr_list[0] in result_:
                dr.append(1)
                # print('1') 
            else:
                dr.append(0)
                # print('0')
        else:
            # aa.append([0]*len(all_snp))
            pass
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1

    return aa_array, dr

## loading cryptic data

In [29]:
df_ = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_ = df_[['ENA_SAMPLE','amikacin','bedaquiline','clofazimine','delamanid','ethambutol','ethionamide','isoniazid','kanamycin','levofloxacin','linezolid','moxifloxacin','rifampicin']]

df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.merge(df, df_, on='ENA_SAMPLE', how='inner')

In [30]:
df.columns

Index(['ENA_RUN', 'UNIQUEID', 'AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE', 'AMI_MIC', 'BDQ_MIC',
       'CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC',
       'AMI_PHENOTYPE_QUALITY', 'BDQ_PHENOTYPE_QUALITY',
       'CFZ_PHENOTYPE_QUALITY', 'DLM_PHENOTYPE_QUALITY',
       'EMB_PHENOTYPE_QUALITY', 'ETH_PHENOTYPE_QUALITY',
       'INH_PHENOTYPE_QUALITY', 'KAN_PHENOTYPE_QUALITY',
       'LEV_PHENOTYPE_QUALITY', 'LZD_PHENOTYPE_QUALITY',
       'MXF_PHENOTYPE_QUALITY', 'RIF_PHENOTYPE_QUALITY',
       'RFB_PHENOTYPE_QUALITY', 'ENA_SAMPLE', 'VCF', 'REGENOTYPED_VCF',
       'amikacin', 'bedaquiline', 'clofazimine', 'delamanid

In [31]:
import pandas as pd
import re

def clean_mic_column(series):
    """
    Remove non-numeric characters and convert to float.
    Keeps digits, decimal points, scientific notation, and sign symbols.
    """
    cleaned = series.astype(str).str.replace(r"[^0-9.\-eE+]", "", regex=True)
    return cleaned.replace("", "nan").astype(float)

mic_cols = ['CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC', 'AMI_MIC', 'BDQ_MIC']

for col in mic_cols:
    df[col] = clean_mic_column(df[col])


In [32]:
import numpy as np
import pandas as pd

def shannon_entropy(series):
    """
    Compute Shannon entropy, max entropy, and number of MIC classes.
    Excludes NaN values.
    """
    s = series.dropna()

    # Count frequencies of each MIC category
    value_counts = s.value_counts(normalize=True)

    # Shannon entropy
    H = -(value_counts * np.log2(value_counts)).sum()

    # number of unique MIC classes
    k = value_counts.shape[0]

    # Max possible entropy
    H_max = np.log2(k) if k > 1 else 0

    return H, H_max, k


mic_cols = ['CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC', 'AMI_MIC', 'BDQ_MIC']

results = []

for col in mic_cols:
    H, H_max, k = shannon_entropy(df[col])
    results.append([col, H, H_max, k])

entropy_table = pd.DataFrame(results, columns=[
    "Drug", "Shannon entropy (H)", "Max entropy (log2 k)", "Num MIC classes (k)"
])

print(entropy_table)


       Drug  Shannon entropy (H)  Max entropy (log2 k)  Num MIC classes (k)
0   CFZ_MIC             1.932456              3.000000                    8
1   DLM_MIC             1.784257              3.000000                    8
2   EMB_MIC             2.417177              3.321928                   10
3   ETH_MIC             2.248237              2.584963                    6
4   INH_MIC             2.680953              3.321928                   10
5   KAN_MIC             1.817434              2.321928                    5
6   LEV_MIC             2.154512              2.807355                    7
7   LZD_MIC             1.915494              3.000000                    8
8   MXF_MIC             2.473755              2.807355                    7
9   RIF_MIC             2.584264              3.169925                    9
10  RFB_MIC             1.527392              2.584963                    6
11  AMI_MIC             1.576502              2.807355                    7
12  BDQ_MIC 

In [21]:
df.columns
import scipy.stats as stats
df = df.dropna()
# KAN vs AMI
rho_kan_ami, p_kan_ami = stats.spearmanr(df['KAN_MIC'], df['AMI_MIC'])
print("Spearman KAN–AMI:")
print("rho =", rho_kan_ami, "p =", p_kan_ami)

# MXF vs LEV
rho_mxf_lev, p_mxf_lev = stats.spearmanr(df['MXF_MIC'], df['LEV_MIC'])
print("\nSpearman MXF–LEV:")
print("rho =", rho_mxf_lev, "p =", f"{p_mxf_lev:.100e}")


Spearman KAN–AMI:
rho = 0.6308777334585122 p = 0.0

Spearman MXF–LEV:
rho = 0.8055377406451715 p = 0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000e+00


In [17]:
df[['KAN_MIC','AMI_MIC']].isna().sum()
len(df[['KAN_MIC','AMI_MIC']].dropna())


11955

In [15]:
for col in mic_cols:
    print(df[col].info())
    print(df[col].value_counts())

<class 'pandas.core.series.Series'>
RangeIndex: 12287 entries, 0 to 12286
Series name: KAN_MIC
Non-Null Count  Dtype  
--------------  -----  
12128 non-null  float64
dtypes: float64(1)
memory usage: 96.1 KB
None
KAN_MIC
2.0     5762
1.0     3541
4.0     1705
16.0     851
8.0      269
Name: count, dtype: int64
<class 'pandas.core.series.Series'>
RangeIndex: 12287 entries, 0 to 12286
Series name: AMI_MIC
Non-Null Count  Dtype  
--------------  -----  
12070 non-null  float64
dtypes: float64(1)
memory usage: 96.1 KB
None
AMI_MIC
0.25     7074
0.50     3510
1.00      604
8.00      425
16.00     328
2.00       94
4.00       35
Name: count, dtype: int64
<class 'pandas.core.series.Series'>
RangeIndex: 12287 entries, 0 to 12286
Series name: MXF_MIC
Non-Null Count  Dtype  
--------------  -----  
12192 non-null  float64
dtypes: float64(1)
memory usage: 96.1 KB
None
MXF_MIC
0.25    4056
0.50    2674
0.12    2200
4.00    1216
0.06     965
1.00     573
2.00     508
Name: count, dtype: int64
<clas

### LZD

In [ ]:
drug = 'LZD'
dr_list = ['linezolid']

gene_list = ['rplC', 'rrl']
# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '<=0.06'])]
df[f'{drug}_MIC'].unique()


array(['0.25', '0.5', '1.0', '2.0', '>2', '0.12', nan, '<=0.03', '0.06',
       '<=0.06', '4.0', '>4'], dtype=object)

In [ ]:
drug = 'LZD'
dr_list = ['linezolid']

gene_list = ['rplC', 'rrl']
# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '<=0.06'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['0.06', '0.12', '0.25', '0.5', '1.0', '2.0', '4.0', '<=0.03','<=0.06', '>2', '>4'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['0.06', '0.12', '0.25', '0.5', '1.0', '2.0', '4.0', '<=0.03','<=0.06', '>2', '>4'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '0.06','<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'                           
    if row[x] == '0.12' :
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '<=0.03':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '>2':
        df_emb.loc[i, f'{x}'] = '4'
    elif row[x] == '>4':
        df_emb.loc[i, f'{x}'] = '4'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796473', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR4829923', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR4830718', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4829042', 'ERR8

100%|██████████| 11854/11854 [00:00<00:00, 23956.00it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(174,)


100%|██████████| 2223/2223 [00:04<00:00, 542.89it/s]


### RIF

In [20]:
drug = 'RIF'

df[f'{drug}_MIC'].unique()
# df['LZD_MIC'].fillna(most_frequent_value, inplace=True)

array(['<=0.06', '0.12', '>4', '4.0', '2.0', '0.25', '0.5', nan, '1.0',
       '>8', '0.06', '<=0.03', '8.0'], dtype=object)

In [21]:
drug = 'RIF'
dr_list = ['rifampicin']

gene_list = ['rpoA', 'rpoB', 'rpoC']
# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '<=0.06'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)
# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '0.06','<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>4' :
        df_emb.loc[i, f'{x}'] = '8'
    elif row[x] == '>8':
        df_emb.loc[i, f'{x}'] = '8'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
        # df_emb.loc[i, f'{x}'] = '0.03'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '<=0.03':
        df_emb.loc[i, f'{x}'] = '0.0625'
          
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
# aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# # encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

# with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_cryptic)
# with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
# with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
#     np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    # np.save(f, np.array(drs_50k))

{'ERR8699178', 'ERR4798048', 'ERR4814125', 'ERR4830669', 'ERR8975538', 'ERR4797661', 'ERR4830739', 'ERR8975101', 'ERR4831168', 'ERR8975464', 'ERR2510536', 'ERR3287537', 'ERR8699842', 'ERR4828293', 'ERR4830942', 'ERR4796611', 'ERR4831914', 'ERR8975496', 'ERR2510524', 'ERR4810782', 'ERR2510330', 'ERR3287718', 'ERR8976038', 'ERR8699141', 'ERR4830077', 'ERR4830335', 'ERR3287817', 'ERR4830872', 'ERR2184356', 'ERR4810576', 'ERR4822633', 'ERR4813461', 'ERR4796897', 'ERR4811503', 'ERR4813277', 'ERR4796770', 'ERR4814011', 'ERR8700278', 'ERR3287785', 'ERR4809278', 'ERR4813343', 'ERR2184443', 'ERR4830383', 'ERR4811786', 'ERR4830189', 'ERR4811349', 'ERR8686170', 'ERR4797396', 'ERR4828162', 'ERR4830907', 'ERR4830102', 'ERR4811657', 'ERR8975097', 'ERR4796862', 'ERR2510719', 'ERR4796725', 'ERR4796729', 'ERR4831261', 'ERR4831846', 'ERR4829705', 'ERR4813236', 'ERR4796876', 'ERR4831139', 'ERR4831256', 'ERR4830433', 'ERR4811413', 'ERR4812426', 'ERR4810841', 'ERR4813179', 'ERR4831398', 'ERR4812611', 'ERR8

100%|██████████| 11769/11769 [00:54<00:00, 217.42it/s]
/tmp/ipykernel_27793/3302620015.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_27793/3302620015.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


In [ ]:

def data_prep_cryptic(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    print(overlap)
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    np.save(f'all_snp_{len(all_snp)}.npy', all_snp)
    print(len(all_snp))
    print(len(overlap))
    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
        else:
            aa.append([0]*len(all_snp))
        
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1
    
    mic_aa = cryptic[cryptic['ENA_RUN'].isin(overlap)]#.iloc[:,14:27]
    # mic_aa = cryptic[cryptic['ENA_RUN'].isin(variants['sample_id'].unique())]#.iloc[:,14:27]
    # print(mic_aa.shape)
    # mic_aa['wgs_id'] = pd.Categorical(mic_aa['ENA_RUN'], categories=variants['sample_id'].unique().tolist(), ordered=True)
    # mic_aa = mic_aa.sort_values('ENA_RUN')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)
    # mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(variants['sample_id'].unique().tolist())
    mic_aa = mic_aa.sort_values(["ENA_RUN"])  ## 'sort' changed to 'sort_values'
    
    return aa_array, mic_aa, all_snp


drug = 'RIF'
dr_list = ['rifampicin']

gene_list = ['rpoA', 'rpoB', 'rpoC']
# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '<=0.06'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)
# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '0.06','<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>4' :
        df_emb.loc[i, f'{x}'] = '8'
    elif row[x] == '>8':
        df_emb.loc[i, f'{x}'] = '8'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
        # df_emb.loc[i, f'{x}'] = '0.03'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '<=0.03':
        df_emb.loc[i, f'{x}'] = '0.0625'
          
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

cryptic = df_emb
aa_array_cryptic, drs_cryptic, all_snp= data_prep_cryptic(cryptic, gene_list,dr_list)

{'ERR4829414', 'ERR4811687', 'ERR4822429', 'ERR4796437', 'ERR4830728', 'ERR4831248', 'ERR4812798', 'ERR4830214', 'ERR2510291', 'ERR4828703', 'ERR4811664', 'ERR4830915', 'ERR4796943', 'ERR8686580', 'ERR2510254', 'ERR4831936', 'ERR4831754', 'ERR4831083', 'ERR4797408', 'ERR4811794', 'ERR3287512', 'ERR2184542', 'ERR4830742', 'ERR4809085', 'ERR4797223', 'ERR4812360', 'ERR5918160', 'ERR4810546', 'ERR2510340', 'ERR4796707', 'ERR8975193', 'ERR4811081', 'ERR4809101', 'ERR5918081', 'ERR4829082', 'ERR4830669', 'ERR4831061', 'ERR4797736', 'ERR4811922', 'ERR2184300', 'ERR4829536', 'ERR4828700', 'ERR4831885', 'ERR4831824', 'ERR4814081', 'ERR4830218', 'ERR4831307', 'ERR4814103', 'ERR4811689', 'ERR2184634', 'ERR4812396', 'ERR4812503', 'ERR2510502', 'ERR4830360', 'ERR4828501', 'ERR8699109', 'ERR4797416', 'ERR4814104', 'ERR4830200', 'ERR4830596', 'ERR4811714', 'ERR3287642', 'ERR8700364', 'ERR4813103', 'ERR2510360', 'ERR4813530', 'ERR4813008', 'ERR4828554', 'ERR4797194', 'ERR4812591', 'ERR4828977', 'ERR4

100%|██████████| 11769/11769 [00:54<00:00, 216.96it/s]
/tmp/ipykernel_40794/3458137031.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_40794/3458137031.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


In [6]:
 all_snp

array(['rpoC-p.Gly594Glu', 'rpoB-p.Ser450Leu', 'rpoB-p.Glu250Gly', ...,
       'rpoC-p.Ala710Gly', 'rpoB-p.Pro483Thr', 'rpoC-p.Thr667Ala'],
      dtype=object)

### ETH


In [16]:
drug = 'ETH'

df[f'{drug}_MIC'].unique()


array(['2.0', '1.0', '4.0', '>8', '0.5', '8.0', '<=0.25', nan],
      dtype=object)

In [ ]:
drug = 'ETH'
dr_list = ['ethionamide']
gene_list = ['ethA', 'inhA']
    
# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '0.25', '<=0.25'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.25'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796473', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR4829923', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR4830718', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4829042', 'ERR8

100%|██████████| 11800/11800 [00:07<00:00, 1677.88it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(1021,)


100%|██████████| 33740/33740 [10:10<00:00, 55.24it/s]


### INH


In [18]:
drug = 'INH'

df[f'{drug}_MIC'].unique()


array(['0.05', '<=0.025', '0.2', '0.1', '>1.6', '1.6', nan, '0.8', '0.4',
       '3.2', '12.8', '>12.8', '6.4'], dtype=object)

In [ ]:
drug = 'INH'
gene_list = ['inhA', 'katG','kasA']
dr_list = ['isoniazid']

# df_emb = df[df[f'{drug}_MIC'].isin(['>1.6','0.8', '0.4', '0.2', '0.1', '0.05', '<=0.025'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>1.6','1.6','0.8', '0.4', '0.2', '0.1', '0.05', '<=0.025'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1.6' :
        df_emb.loc[i, f'{x}'] = '3.2'
    elif row[x] == '<=0.025':
        # df_emb.loc[i, f'{x}'] = '0.0125'
        df_emb.loc[i, f'{x}'] = '0.025'
    elif row[x] == '>12.8':
        df_emb.loc[i, f'{x}'] = '3.2'
    elif row[x] == '12.8':
        df_emb.loc[i, f'{x}'] = '3.2'
    elif row[x] == '>6.4':
        df_emb.loc[i, f'{x}'] = '3.2'
        # df_emb.loc[i, f'{x}'] = '0.025'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)


# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR8975826', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796473', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR4829923', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR4830718', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4

100%|██████████| 11736/11736 [00:36<00:00, 319.27it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(1050,)


100%|██████████| 93964/93964 [1:15:53<00:00, 20.63it/s]


### RFB -- skipped


In [ ]:
drug = 'RFB'
res_thresh = 0.12
dr_list = ['rifabutin']
gene_list = ['rpoA', 'rpoB', 'rpoC']
# df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12', '<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>2' :
        df_emb.loc[i, f'{x}'] = '4'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.03125'
        
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

### MXF


In [20]:
drug = 'MXF'

df[f'{drug}_MIC'].unique()


array(['0.25', '0.5', '0.12', '2.0', '4.0', '1.0', '>4', '<=0.06', nan],
      dtype=object)

In [42]:
drug = 'MXF'
dr_list = ['moxifloxacin']
gene_list = ['gyrA', 'gyrB']
res_thresh = 1

# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4', '2', '1', '0.5', '0.25', '0.12', '<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>4' :
        df_emb.loc[i, f'{x}'] = '8'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float') 
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))        
    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR8975826', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796473', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR4829923', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR4830718', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4

100%|██████████| 11860/11860 [01:40<00:00, 117.91it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(879,)


100%|██████████| 122756/122756 [4:27:51<00:00,  7.64it/s] 


### KAN 


In [21]:
drug = 'KAN'

df[f'{drug}_MIC'].unique()

array(['2.0', '<=1', '>16', '16.0', '4.0', nan, '8.0'], dtype=object)

In [43]:
drug = 'KAN'
dr_list = ['kanamycin']
gene_list = ['rrs', 'eis']
res_thresh = 4

# df_emb = df[df[f'{drug}_MIC'].isin(['>16','16.0', '8.0', '4.0', '2.0', '<=1'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>16' :
        df_emb.loc[i, f'{x}'] = '32'
    elif row[x] == '<=1':
        df_emb.loc[i, f'{x}'] = '1.0'

        # df_emb.loc[i, f'{x}'] = '0.5'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR8975826', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796473', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4829042', 'ERR8686578', 'ERR4

100%|██████████| 11797/11797 [00:39<00:00, 301.74it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(310,)


100%|██████████| 123648/123648 [1:36:22<00:00, 21.38it/s]


### LEV


In [22]:
drug = 'LEV'

df[f'{drug}_MIC'].unique()

array(['0.25', '1.0', '0.5', '<=0.12', '2.0', '8.0', '>8', '4.0', nan],
      dtype=object)

In [59]:
drug = 'LEV'
# gene_list = ['pgi', 'fadE4', 'php', 'cyp132', 'pckA', 'rpmB1', 'pfkB', 'acg', 'ctpF', 'cyp132', 'pckA', 'pfk']
# gene_list = np.unique(variants['gene'])
dr_list = ['levofloxacin']
gene_list = ['gyrA', 'gyrB']
res_thresh = 1

# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '0.25', '<=0.12'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
        # df_emb.loc[i, f'{x}'] = '0.06'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')

In [60]:
df_emb

,ENA_RUN,UNIQUEID,AMI_BINARY_PHENOTYPE,BDQ_BINARY_PHENOTYPE,CFZ_BINARY_PHENOTYPE,DLM_BINARY_PHENOTYPE,EMB_BINARY_PHENOTYPE,ETH_BINARY_PHENOTYPE,INH_BINARY_PHENOTYPE,KAN_BINARY_PHENOTYPE,...,clofazimine,delamanid,ethambutol,ethionamide,isoniazid,kanamycin,levofloxacin,linezolid,moxifloxacin,rifampicin
0,ERR4810489,site.02.subj.0001.lab.2014222001.iso.1,S,NaN,S,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
1,ERR4810491,site.02.subj.0002.lab.2014222005.iso.1,S,S,S,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
2,ERR4810493,site.02.subj.0004.lab.2014222010.iso.1,S,S,S,NaN,S,I,S,S,...,0,0,0,0,0,0,0,0,0,0
3,ERR4810494,site.02.subj.0005.lab.2014222011.iso.1,S,S,S,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
4,ERR4810495,site.02.subj.0006.lab.2014222013.iso.1,S,S,S,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12282,ERR8699231,site.20.subj.SCH8604398.lab.YA00134621.iso.1,S,S,S,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
12283,ERR8699764,site.20.subj.SCH8604399.lab.YA00134971.iso.1,S,S,S,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
12284,ERR8699789,site.20.subj.SCH8608291.lab.YA00135762.iso.1,S,S,S,S,R,R,R,S,...,0,0,1,1,1,0,1,0,1,1
12285,ERR8699232,site.20.subj.SCH8608295.lab.YA00134623.iso.1,S,S,S,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0


In [44]:
drug = 'LEV'
# gene_list = ['pgi', 'fadE4', 'php', 'cyp132', 'pckA', 'rpmB1', 'pfkB', 'acg', 'ctpF', 'cyp132', 'pckA', 'pfk']
# gene_list = np.unique(variants['gene'])
dr_list = ['levofloxacin']
gene_list = ['gyrA', 'gyrB']
res_thresh = 1

# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '0.25', '<=0.12'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
        # df_emb.loc[i, f'{x}'] = '0.06'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796473', 'ERR4796755', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR4829923', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR4830718', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4829042', 'ERR8686578', 'ERR4

100%|██████████| 11829/11829 [01:38<00:00, 120.00it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(882,)


100%|██████████| 122787/122787 [4:22:40<00:00,  7.79it/s] 


### AMI


In [23]:
drug = 'AMI'

df[f'{drug}_MIC'].unique()

array(['<=0.25', '0.5', '1.0', '>8', '4.0', '2.0', '8.0', nan, '>16',
       '16.0'], dtype=object)

In [45]:
# gene_list = ['pgi', 'fadE4', 'php', 'cyp132', 'pckA', 'rpmB1', 'pfkB', 'acg', 'ctpF', 'cyp132', 'pckA', 'pfk']
# gene_list = np.unique(variants['gene'])
dr_list = ['amikacin']
drug = 'AMI'
# gene_list = np.unique(variants['gene'])
gene_list = ['rrs', 'eis']

res_thresh = 1
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '<=0.25'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>16' :
        df_emb.loc[i, f'{x}'] = '16'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.25'
        # df_emb.loc[i, f'{x}'] = '0.125'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))        
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4829042', 'ERR8686578', 'ERR4830102', 'ERR4796829', 'ERR4

100%|██████████| 11738/11738 [00:38<00:00, 305.52it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(308,)


100%|██████████| 123707/123707 [1:36:08<00:00, 21.44it/s]


### BDQ



In [24]:
drug = 'BDQ'

df[f'{drug}_MIC'].unique()

array([nan, '0.06', '0.03', '0.25', '<=0.015', '0.5', '0.12', '2.0', '>2',
       '0.015', '<=0.008', '>1', '1.0'], dtype=object)

In [46]:
drug = 'BDQ'
gene_list = [ 'Rv0678', 'atpE', 'pepQ']
res_thresh = 0.25
dr_list = ['bedaquiline']

# df_emb = df[df[f'{drug}_MIC'].isin(['>1', '1.0', '0.5', '0.25', '0.12', '0.06', '0.03', '<=0.015'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1' :
        df_emb.loc[i, f'{x}'] = '2'
    elif row[x] == '>2':
        df_emb.loc[i, f'{x}'] = '2'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.03125'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'
    elif row[x] == '<=0.008':
        df_emb.loc[i, f'{x}'] = '0.015625'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR8975826', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR4829923', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR4830718', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4829042', 'ERR8

100%|██████████| 11743/11743 [00:01<00:00, 7717.21it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(276,)


100%|██████████| 11680/11680 [00:56<00:00, 207.13it/s]


### CFZ


In [15]:
drug = 'CFZ'

df[f'{drug}_MIC'].unique()

array(['<=0.06', '0.12', '0.25', '0.5', '2.0', '1.0', '4.0', nan, '>4',
       '<=0.03', '0.06', '>2'], dtype=object)

In [17]:
drug = 'CFZ'
dr_list = ['clofazimine']
# gene_list = np.unique(variants['gene'])
gene_list = [ 'Rv0678', 'pepQ']
res_thresh = 0.25

# df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12', '0.06', '0.03', '<=0.015'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1' :
        df_emb.loc[i, f'{x}'] = '2'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '>2':
        df_emb.loc[i, f'{x}'] = '4'
    elif row[x] == '>4':
        df_emb.loc[i, f'{x}'] = '4'
    elif row[x] == '<=0.03':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '<=0.03':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
cryptic = df_emb

variants = pd.read_csv('../variants_full.csv')
variants = variants[variants['gene'] != 'PPE35']
variants = variants[variants['type'] != 'synonymous_variant']
variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
# variants = variants[variants['drugs'].isin(['ethambutol'])]
variants = variants[variants['gene'].isin(gene_list)]
variants = variants[variants['sample_id'].isin(overlap)]
variants['SNP'] = variants['gene'] + '-'+ variants['change']

print(variants['SNP'].unique().shape)

def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
    query_dict = dict(zip(query_list, values_list))
    output_list = [query_dict.get(snp, 0) for snp in set_list]
    return output_list

aa = []
dr = []

all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table

variants = pd.read_csv('../variants_full.csv')
variants = variants[variants['gene'] != 'PPE35']
variants = variants[variants['type'] != 'synonymous_variant']
variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
variants['SNP'] = variants['gene'] + '-'+ variants['change']
variants = variants[variants['SNP'].isin(all_snp)]
# certify we want 50k data that doesn't overlap with cryptic
overlap = variants[~variants['sample_id'].isin(cryptic['ENA_RUN'])]
overlap = overlap['sample_id'].unique()

for x in tqdm(overlap):
    if x in variants['sample_id'].tolist():
        aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
        result_ = transform_array(variants[variants['sample_id']==x]['drugs'].unique())
        if dr_list[0] in result_:
            dr.append(1)
        else:
            dr.append(0)
    else:
        pass
aa_array = np.array(aa)
aa_array[aa_array < 0.8] = 0
aa_array[aa_array >= 0.8] = 1


(197,)


100%|██████████| 7921/7921 [00:27<00:00, 289.90it/s]


In [19]:
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(dr))


In [100]:
aa_array_= np.load(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy')
print(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy')

./generated_data18122024/all_sample_snps_50k_CFZ.npy


In [98]:
aa_array_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [48]:
drug = 'CFZ'
dr_list = ['clofazimine']
# gene_list = np.unique(variants['gene'])
gene_list = [ 'Rv0678', 'pepQ']
res_thresh = 0.25

# df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12', '0.06', '0.03', '<=0.015'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
# df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12', '0.06', '0.03', '<=0.015'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1' :
        df_emb.loc[i, f'{x}'] = '2'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'
    elif row[x] == '>2':
        df_emb.loc[i, f'{x}'] = '4'
    elif row[x] == '>4':
        df_emb.loc[i, f'{x}'] = '4'
    elif row[x] == '<=0.03':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR4829923', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR2184288', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4829042', 'ERR8686578', 'ERR4830102', 'ERR4796829', 'ERR4

100%|██████████| 11718/11718 [00:01<00:00, 9462.43it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(197,)


100%|██████████| 7921/7921 [00:28<00:00, 282.16it/s]


### DLM


In [51]:
drug = 'DLM'

df[f'{drug}_MIC'].unique()

array(['<=0.015', nan, '0.03', '0.06', '0.12', '>1', '0.5', '0.25', '1.0',
       '<=0.008', '0.015', '>0.5'], dtype=object)

In [ ]:

drug = 'DLM'
dr_list = ['delamanid']
res_thresh = 0.12
gene_list = ['fgd1','fbiA','fbiB','fbiC','fbiD','ddn']
# gene_list = variants['gene'].unique()

# df_emb = df[df[f'{drug}_MIC'].isin(['>0.5', '0.5', '0.25', '0.12', '0.06', '0.03', '0.015', '<=0.015'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>0.5' :
        df_emb.loc[i, f'{x}'] = '1'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'        
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.03125'
    elif row[x] == '0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'        
    elif row[x] == '>1':
        df_emb.loc[i, f'{x}'] = '1'      
    elif row[x] == '<=0.008':
        df_emb.loc[i, f'{x}'] = '0.015625'
        
                
                
             
                
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')

In [57]:
df_emb[f'{drug}_MIC'].unique()

array([0.0625  , 0.03125 , 0.125   , 1.      , 0.5     , 0.25    ,
       0.015625])

In [ ]:
drug = 'DLM'
dr_list = ['delamanid']
res_thresh = 0.12
gene_list = ['fgd1','fbiA','fbiB','fbiC','fbiD','ddn']
# gene_list = variants['gene'].unique()

# df_emb = df[df[f'{drug}_MIC'].isin(['>0.5', '0.5', '0.25', '0.12', '0.06', '0.03', '0.015', '<=0.015'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>0.5' :
        df_emb.loc[i, f'{x}'] = '1'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'        
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.03125'
    elif row[x] == '0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'        
    elif row[x] == '>1':
        df_emb.loc[i, f'{x}'] = '1'      
    elif row[x] == '<=0.008':
        df_emb.loc[i, f'{x}'] = '0.015625'
        
                
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR8975826', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4813899', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796473', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR4812585', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4829042', 'ERR8686578', 'ERR4

100%|██████████| 11594/11594 [00:11<00:00, 1000.48it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(1263,)


100%|██████████| 50467/50467 [22:39<00:00, 37.12it/s]


### EMB

In [28]:
drug = 'EMB'

df[f'{drug}_MIC'].unique()

array(['1.0', '2.0', '0.5', '4.0', '>8', '8.0', nan, '0.25', '<=0.06',
       '0.12', '16.0', '>32', '32.0', '<=0.25'], dtype=object)

In [50]:
drug = 'EMB'
dr_list = ['ethambutol']
res_thresh = 8
gene_list = ['embB', 'embA', 'embC']
# gene_list = variants['gene'].unique()
# df_emb = df[df['EMB_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
df_emb = df.copy().dropna(subset=[f'{drug}_MIC'], inplace=False)

# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = 'EMB_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.25'
    elif row[x] == '>32':
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '32.0':
        df_emb.loc[i, f'{x}'] = '16'  
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.25'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.25'    
             
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')

cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

{'ERR2510642', 'ERR8975826', 'ERR4828670', 'ERR4797476', 'ERR4830181', 'ERR4830150', 'ERR4812027', 'ERR3287478', 'ERR4809145', 'ERR4830423', 'ERR4830821', 'ERR3287321', 'ERR4809173', 'ERR4797074', 'ERR4810625', 'ERR8975535', 'ERR4831081', 'ERR2510332', 'ERR3287319', 'ERR4810827', 'ERR8975634', 'ERR4796705', 'ERR2184230', 'ERR2510287', 'ERR4830497', 'ERR4829071', 'ERR4796604', 'ERR3287378', 'ERR4813434', 'ERR4830603', 'ERR2510684', 'ERR4829736', 'ERR4828371', 'ERR2184517', 'ERR4813956', 'ERR4808955', 'ERR4831748', 'ERR8665927', 'ERR8699089', 'ERR3287504', 'ERR2184637', 'ERR4831072', 'ERR4814070', 'ERR8975051', 'ERR8975510', 'ERR5917761', 'ERR4828422', 'ERR4830256', 'ERR4796473', 'ERR4796755', 'ERR4814027', 'ERR2510263', 'ERR4812013', 'ERR4808932', 'ERR4829923', 'ERR8975911', 'ERR4831939', 'ERR4830218', 'ERR4797224', 'ERR2184288', 'ERR4830718', 'ERR8975586', 'ERR4812942', 'ERR4811004', 'ERR4828789', 'ERR4813994', 'ERR8686191', 'ERR4810955', 'ERR4810613', 'ERR4829042', 'ERR8686578', 'ERR4

100%|██████████| 11823/11823 [01:20<00:00, 145.99it/s]
/tmp/ipykernel_129167/2032093787.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_129167/2032093787.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(1810,)


100%|██████████| 122294/122294 [3:29:36<00:00,  9.72it/s] 


# sanity check with the EMB

In [21]:
remove_from_50k = ['ERR1213824', 'ERR1213825', 'ERR1213826', 'ERR1213827', 'ERR1213828', 'ERR1213829', 'ERR1213830', 'ERR1213831', 'ERR1213832', 'ERR1213833', 'ERR1213834', 'ERR1213835', 'ERR1213836', 'ERR1213837', 'ERR1213838', 'ERR1213839', 'ERR1213840', 'ERR1213841', 'ERR1213842', 'ERR1213843', 'ERR1213844', 'ERR1213845', 'ERR1213846', 'ERR1213847', 'ERR1213848', 'ERR1213849', 'ERR1213850', 'ERR1213851', 'ERR1213852', 'ERR1213853', 'ERR1213854', 'ERR1213855', 'ERR1213856', 'ERR1213857', 'ERR1213858', 'ERR1213859', 'ERR1213860', 'ERR1213861', 'ERR1213862', 'ERR1213863', 'ERR1213864', 'ERR1213865', 'ERR1213866', 'ERR1213867', 'ERR1213868', 'ERR1213869', 'ERR1213870', 'ERR1213871', 'ERR1213872', 'ERR1213873', 'ERR1213874', 'ERR1213875', 'ERR1213876', 'ERR1213877', 'ERR1213878', 'ERR1213879', 'ERR1213880', 'ERR1213881', 'ERR1213882', 'ERR1213883', 'ERR1213884', 'ERR1213885', 'ERR1213886', 'ERR1213887', 'ERR1213888', 'ERR1213889', 'ERR1213890', 'ERR1213891', 'ERR1213892', 'ERR1213893', 'ERR1213894', 'ERR1213895', 'ERR1213896', 'ERR1213897', 'ERR1213898', 'ERR1213899', 'ERR1213900', 'ERR1213901', 'ERR1213902', 'ERR1213903', 'ERR1213904', 'ERR1213905', 'ERR1213906', 'ERR1213907', 'ERR1213908', 'ERR1213909', 'ERR1213910', 'ERR1213911', 'ERR1213912', 'ERR1213913', 'ERR1213914', 'ERR1213915', 'ERR1213916', 'ERR1213917', 'ERR1213918', 'ERR1213919', 'ERR1213920', 'ERR1213921', 'ERR1213922', 'ERR1213923', 'ERR1213924', 'ERR1213925', 'ERR1213926', 'ERR1213927', 'ERR1213928', 'ERR1213929', 'ERR1213930', 'ERR1213931', 'ERR1213932', 'ERR1213933', 'ERR1213934', 'ERR1213935', 'ERR1213936', 'ERR1213937', 'ERR1213938', 'ERR1213939', 'ERR1213940', 'ERR1213941', 'ERR1213942', 'ERR1213943', 'ERR1213944', 'ERR1213945', 'ERR1213946', 'ERR1213947', 'ERR1213948', 'ERR1213949', 'ERR1213950']

In [22]:
variants = pd.read_csv('../variants_full.csv')


In [23]:
variants[~variants['sample_id'].isin(remove_from_50k)]

,sample_id,genome_pos,gene,change,freq,type,sublin,drtype,drugs
0,ERR9787047,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
1,ERR9787047,7362,gyrA,p.Glu21Gln,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
2,ERR9787047,7585,gyrA,p.Ser95Thr,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
3,ERR9787047,8493,gyrA,p.Leu398Phe,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
4,ERR9787047,9143,gyrA,c.1842T>C,1.000000,synonymous_variant,lineage6.1.1,Sensitive,NaN
...,...,...,...,...,...,...,...,...,...
7420991,ERR718460,4328399,ethA,c.-926C>T,0.986486,upstream_gene_variant,lineage2.2.1,Other,NaN
7420992,ERR718460,4338365,whiB6,p.Cys53Arg,1.000000,missense_variant,lineage2.2.1,Other,NaN
7420993,ERR718460,4338595,whiB6,c.-75delG,1.000000,upstream_gene_variant,lineage2.2.1,Other,NaN
7420994,ERR718460,4407588,gid,c.615A>G,1.000000,synonymous_variant,lineage2.2.1,Other,NaN


#### 50k with samples removed

In [24]:

def data_prep_50k_mod(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
### 50k data reload
    variants = pd.read_csv('../variants_full.csv')
    remove_from_50k = ['ERR1213824', 'ERR1213825', 'ERR1213826', 'ERR1213827', 'ERR1213828', 'ERR1213829', 'ERR1213830', 'ERR1213831', 'ERR1213832', 'ERR1213833', 'ERR1213834', 'ERR1213835', 'ERR1213836', 'ERR1213837', 'ERR1213838', 'ERR1213839', 'ERR1213840', 'ERR1213841', 'ERR1213842', 'ERR1213843', 'ERR1213844', 'ERR1213845', 'ERR1213846', 'ERR1213847', 'ERR1213848', 'ERR1213849', 'ERR1213850', 'ERR1213851', 'ERR1213852', 'ERR1213853', 'ERR1213854', 'ERR1213855', 'ERR1213856', 'ERR1213857', 'ERR1213858', 'ERR1213859', 'ERR1213860', 'ERR1213861', 'ERR1213862', 'ERR1213863', 'ERR1213864', 'ERR1213865', 'ERR1213866', 'ERR1213867', 'ERR1213868', 'ERR1213869', 'ERR1213870', 'ERR1213871', 'ERR1213872', 'ERR1213873', 'ERR1213874', 'ERR1213875', 'ERR1213876', 'ERR1213877', 'ERR1213878', 'ERR1213879', 'ERR1213880', 'ERR1213881', 'ERR1213882', 'ERR1213883', 'ERR1213884', 'ERR1213885', 'ERR1213886', 'ERR1213887', 'ERR1213888', 'ERR1213889', 'ERR1213890', 'ERR1213891', 'ERR1213892', 'ERR1213893', 'ERR1213894', 'ERR1213895', 'ERR1213896', 'ERR1213897', 'ERR1213898', 'ERR1213899', 'ERR1213900', 'ERR1213901', 'ERR1213902', 'ERR1213903', 'ERR1213904', 'ERR1213905', 'ERR1213906', 'ERR1213907', 'ERR1213908', 'ERR1213909', 'ERR1213910', 'ERR1213911', 'ERR1213912', 'ERR1213913', 'ERR1213914', 'ERR1213915', 'ERR1213916', 'ERR1213917', 'ERR1213918', 'ERR1213919', 'ERR1213920', 'ERR1213921', 'ERR1213922', 'ERR1213923', 'ERR1213924', 'ERR1213925', 'ERR1213926', 'ERR1213927', 'ERR1213928', 'ERR1213929', 'ERR1213930', 'ERR1213931', 'ERR1213932', 'ERR1213933', 'ERR1213934', 'ERR1213935', 'ERR1213936', 'ERR1213937', 'ERR1213938', 'ERR1213939', 'ERR1213940', 'ERR1213941', 'ERR1213942', 'ERR1213943', 'ERR1213944', 'ERR1213945', 'ERR1213946', 'ERR1213947', 'ERR1213948', 'ERR1213949', 'ERR1213950']
    variants = variants[~variants['sample_id'].isin(remove_from_50k)]
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = variants[variants['SNP'].isin(all_snp)]
    # certify we want 50k data that doesn't overlap with cryptic
    overlap = variants[~variants['sample_id'].isin(cryptic['ENA_RUN'])]
    overlap = overlap['sample_id'].unique()

    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
            result_ = transform_array(variants[variants['sample_id']==x]['drugs'].unique())
            # print(result_)
            # print(dr_list[0])
            if dr_list[0] in result_:
                dr.append(1)
                # print('1') 
            else:
                dr.append(0)
        else:
            # aa.append([0]*len(all_snp))
            pass
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1

    return aa_array, dr

In [25]:
drug = 'EMB'
dr_list = ['ethambutol']
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

gene_list = ['embB', 'embA', 'embC']
df_emb = df[df['EMB_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = 'EMB_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.125'
        
df_emb['EMB_MIC'] = df_emb['EMB_MIC'].astype('float') 
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
# aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k_mod(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

# with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_cryptic)
# with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}_checks.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}_checks.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

(0,)
0


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

#### removed samples

In [ ]:

def data_prep_50k_mod(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    # np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
### 50k data reload
    variants = pd.read_csv('../variants_full.csv')
    remove_from_50k = ['ERR1213824', 'ERR1213825', 'ERR1213826', 'ERR1213827', 'ERR1213828', 'ERR1213829', 'ERR1213830', 'ERR1213831', 'ERR1213832', 'ERR1213833', 'ERR1213834', 'ERR1213835', 'ERR1213836', 'ERR1213837', 'ERR1213838', 'ERR1213839', 'ERR1213840', 'ERR1213841', 'ERR1213842', 'ERR1213843', 'ERR1213844', 'ERR1213845', 'ERR1213846', 'ERR1213847', 'ERR1213848', 'ERR1213849', 'ERR1213850', 'ERR1213851', 'ERR1213852', 'ERR1213853', 'ERR1213854', 'ERR1213855', 'ERR1213856', 'ERR1213857', 'ERR1213858', 'ERR1213859', 'ERR1213860', 'ERR1213861', 'ERR1213862', 'ERR1213863', 'ERR1213864', 'ERR1213865', 'ERR1213866', 'ERR1213867', 'ERR1213868', 'ERR1213869', 'ERR1213870', 'ERR1213871', 'ERR1213872', 'ERR1213873', 'ERR1213874', 'ERR1213875', 'ERR1213876', 'ERR1213877', 'ERR1213878', 'ERR1213879', 'ERR1213880', 'ERR1213881', 'ERR1213882', 'ERR1213883', 'ERR1213884', 'ERR1213885', 'ERR1213886', 'ERR1213887', 'ERR1213888', 'ERR1213889', 'ERR1213890', 'ERR1213891', 'ERR1213892', 'ERR1213893', 'ERR1213894', 'ERR1213895', 'ERR1213896', 'ERR1213897', 'ERR1213898', 'ERR1213899', 'ERR1213900', 'ERR1213901', 'ERR1213902', 'ERR1213903', 'ERR1213904', 'ERR1213905', 'ERR1213906', 'ERR1213907', 'ERR1213908', 'ERR1213909', 'ERR1213910', 'ERR1213911', 'ERR1213912', 'ERR1213913', 'ERR1213914', 'ERR1213915', 'ERR1213916', 'ERR1213917', 'ERR1213918', 'ERR1213919', 'ERR1213920', 'ERR1213921', 'ERR1213922', 'ERR1213923', 'ERR1213924', 'ERR1213925', 'ERR1213926', 'ERR1213927', 'ERR1213928', 'ERR1213929', 'ERR1213930', 'ERR1213931', 'ERR1213932', 'ERR1213933', 'ERR1213934', 'ERR1213935', 'ERR1213936', 'ERR1213937', 'ERR1213938', 'ERR1213939', 'ERR1213940', 'ERR1213941', 'ERR1213942', 'ERR1213943', 'ERR1213944', 'ERR1213945', 'ERR1213946', 'ERR1213947', 'ERR1213948', 'ERR1213949', 'ERR1213950']
    variants = variants[variants['sample_id'].isin(remove_from_50k)]
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = variants[variants['SNP'].isin(all_snp)]
    # certify we want 50k data that doesn't overlap with cryptic
    overlap = variants[~variants['sample_id'].isin(cryptic['ENA_RUN'])]
    overlap = overlap['sample_id'].unique()
    print("len(overlap)",len(overlap))

    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
            
            result_ = transform_array(variants[variants['sample_id']==x]['drugs'].unique())
            # print(result_)
            # print(dr_list[0])
            if dr_list[0] in result_:
                dr.append(1)
                # print('1') 
            else:
                dr.append(0)
        else:
            # aa.append([0]*len(all_snp))
            pass
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1

    return aa_array, dr

In [ ]:
drug = 'EMB'
dr_list = ['ethambutol']
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

gene_list = ['embB', 'embA', 'embC']
df_emb = df[df['EMB_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = 'EMB_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.125'
        
df_emb['EMB_MIC'] = df_emb['EMB_MIC'].astype('float') 
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
# aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k_mod(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

# with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_cryptic)
# with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}_checks-removed.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}_checks-removed.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/tmp/ipykernel_52736/1811954684.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

(1710,)
1710
len(overlap) 125


100%|██████████| 125/125 [00:00<00:00, 760.72it/s]


In [ ]:
test_values = ['8', '4', '≤1', '≤1', '2', '≤1', '≤1', '2', '2', '≤1', '≤1', '4', '2', '4', '≤1', '8', '2', '2', '4', '4', '8', '≤1', '≤1', '≤1', '2', '>','8', '4', '2', '4', '2', '≤1', '2', '2', '≤1', '4', '>','8', '2', '≤1', '4', '4', '8', '4', '2', '≤1', '≤1', '≤1', '≤1', '4', '4', '≤1', '>','8', '≤1', '≤1', '2', '2', '≤1', '≤1', '2', '2', '4', '≤1', '≤1', '2', '4', '≤1', '8', '2', '2', '4', '2', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '2', '4', '≤1', '4', '≤1', '≤1', '≤1', '≤1', '4', '2', '≤1', '≤1', '≤1', '2', '≤1', '≤1', '≤1', '≤1', '≤1', '2', '≤1', '≤1', '8', '2', '≤1', '8', '2', '4', '2', '≤1', '≤1', '2', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '2', '≤1', '≤1', '≤1', '≤1', '≤1', '>','8', '8', '4', '4']
for i, x in enumerate(test_values):
    if x == '≤1':
        test_values[i] = '1'